In [1]:
import pandas as pd
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
def load_data():
    """Load dataset from CSV file and split into training and testing sets."""

    dataf = pd.read_csv(r'C:\Users\AnerghaKMohan\Documents\LLM\AIDetect\labeled_data.csv')
    trains_text, tests_text, trains_labels, tests_labels = train_test_split(
        dataf['message'],
        dataf['class'],
        test_size=0.2
    )

    return trains_text, tests_text, trains_labels, tests_labels

In [3]:
load_data()

(9541     ong I've been thinking about getting a small d...
 11508    Last call challenger is a demon that runs near...
 12539                 made sure of that from the age of 15
 3952                     can't say the same for a car ride
 18879    if he couldve only focused that interest into ...
                                ...                        
 13621        No I just can’t put the second one above 60hz
 10190               Like a pair of headphones or something
 11707    i know all u youngens like to play shit games ...
 11008    obviously the feds don't care about some com s...
 9088                          u need to use cat fish heads
 Name: message, Length: 16380, dtype: object,
 4398     nigga said turkish delight nothing in that shi...
 6930         yeah tell me more yrx the military strategist
 7378     Also why do you have a picture of me with cum ...
 1021     That's rather funny. I remember reading shit l...
 3942                      Perk you dont even have a p

In [4]:
def tokenize_data(trained_text, tested_text):
    """Tokenize text data using Tokenizer from Keras and pad sequences to a fixed length."""

    tokenizer_data = Tokenizer(
        num_words=5000
    )
    tokenizer_data.fit_on_texts(
        trained_text
    )
    trained_sequences = pad_sequences(
        tokenizer_data.texts_to_sequences(trained_text),
        maxlen=100
    )
    tested_sequences = pad_sequences(
        tokenizer_data.texts_to_sequences(tested_text),
        maxlen=100
    )

    return tokenizer_data, trained_sequences, tested_sequences

In [6]:
def define_model():
    """Define a convolutional neural network model using Keras."""

    input_layer = Input(
        shape=(100,)
    )
    embedding_layer = Embedding(
        input_dim=5000,
        output_dim=50
    )(input_layer)
    conv_layer = Conv1D(
        filters=128,
        kernel_size=5,
        activation='relu'
    )(embedding_layer)
    pooling_layer = MaxPooling1D(pool_size=5)(conv_layer)
    flatten_layer = Flatten()(pooling_layer)
    output_layer = Dense(
        units=1,
        activation='sigmoid'
    )(flatten_layer)
    model_defined = Model(
        inputs=input_layer,
        outputs=output_layer
    )
    model_defined.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    return model_defined


In [7]:
def train_model(model_training,sequences_training,labels_training,sequences_testing,labels_testing):
    """Train the convolutional neural network model."""

    model_training.fit(
        sequences_training,
        labels_training,
        epochs=1,
        batch_size=1, # Works better, dont be a puss
        validation_data=(sequences_testing, labels_testing)
    )


In [8]:
def save_model(saved_model):
    """Save the trained convolutional neural network model to a file."""

    saved_model.save(
        'Models/ai_detection_model.h5'
    )

In [11]:
def classify_input(model_classify, tokenizer_classify):
    """Classify a text string input as AI or human using the trained model and tokenizer."""

    while True:
        user_input = input(
            'Enter a text string to classify (or "exit" to quit): '
        )

        if user_input.lower() == 'exit':
            break

        if user_input.lower() == '':
            continue

        sequence = pad_sequences(
            tokenizer_classify.texts_to_sequences([user_input]),
            maxlen=100
        )
        prediction = model_classify.predict(sequence)[0][0]

        if prediction > 0.5:
            print("AI generated")
        else:
            print("Human generated")

In [12]:
if __name__ == '__main__':
    train_text, test_text, train_labels, test_labels = load_data()
    tokenizer, train_sequences, test_sequences = tokenize_data(
        train_text,
        test_text
    )
    
    model = define_model()
    
    train_model(model,
                train_sequences,
                train_labels,
                test_sequences,
                test_labels
    )
    save_model(
        model
    )
    print("save model--------------")
    classify_input(
        model,
        tokenizer
    )

16380/16380 ━━━━━━━━━━━━━━━━━━━━ 65s 4ms/step - accuracy: 0.9952 - loss: 0.0280 - val_accuracy: 0.9968 - val_loss: 0.0161


save model--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
AI generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
AI generated
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Human generated
